In [1]:
import cv2
import numpy as np
import threading
import queue
import time

BUFFER_SIZE = 5  # Number of frames to buffer between producer and consumer
DISPLAY_FPS = 30 # Target FPS for display. Setting this too high might cause stuttering.

hog = cv2.HOGDescriptor()
hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())

# Queues for inter-thread communication
frame_queue = queue.Queue(maxsize=BUFFER_SIZE)
processed_frame_queue = queue.Queue(maxsize=BUFFER_SIZE)

# Event to signal when the video reading is done
video_finished = threading.Event()
processing_finished = threading.Event()

# --- Producer Thread: Reads frames from video ---
def frame_reader(video_path):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error: Could not open video {video_path}")
        video_finished.set()
        return

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame_queue.put(frame)
    
    cap.release()
    video_finished.set()
    print("Frame reader finished.")

# --- Consumer Thread: Processes frames ---
def frame_processor():
    while True:
        if video_finished.is_set() and frame_queue.empty():
            break

        try:
            # This prevents the thread from blocking indefinitely if the producer is slow
            # or has finished and there are no more frames.
            frame = frame_queue.get(timeout=0.1) 
        except queue.Empty:
            continue # No frame available, try again

        frame_resized = cv2.resize(frame, (640, 480))

        boxes, weights = hog.detectMultiScale(frame_resized, winStride=(8,8), scale=1.02, padding=(4,4))

        for i, (x,y,w,h) in enumerate(boxes):
            if weights[i] < 0.13:
                continue
            #elif weights[i] < 0.3 and weights[i] > 0.13:
            #    cv2.rectangle(frame_resized, (x, y), (x+w, y+h), (0, 0, 255), 2)
            #elif weights[i] < 0.7 and weights[i] > 0.3:
            #    cv2.rectangle(frame_resized, (x, y), (x+w, y+h), (50, 122, 255), 2)
            elif weights[i] > 0.7:
                cv2.rectangle(frame_resized, (x, y), (x+w, y+h), (0, 255, 0), 2)

        cv2.putText(frame_resized, 'High', (10, 25), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        #cv2.putText(frame_resized, 'Moderate', (10, 45), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (50, 122, 255), 2)
        #cv2.putText(frame_resized, 'Low', (10, 65), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        
        processed_frame_queue.put(frame_resized)
        frame_queue.task_done()
    
    processing_finished.set() # Signal that processing is complete
    print("Frame processor finished.")

def main():
    video_path = '../853889-hd_1920_1080_25fps.mp4'

    # Start the producer thread
    reader_thread = threading.Thread(target=frame_reader, args=(video_path,))
    reader_thread.daemon = True # Allow the main program to exit even if this thread is running
    reader_thread.start()

    # Start the consumer thread
    processor_thread = threading.Thread(target=frame_processor)
    processor_thread.daemon = True
    processor_thread.start()

    last_display_time = time.time()

    while True:
        if video_finished.is_set() and processing_finished.is_set() and processed_frame_queue.empty():
            break
        
        try:
            # Get processed frame for display
            frame_to_display = processed_frame_queue.get(timeout=0.1) 
        except queue.Empty:
            # If the queue is empty, wait a bit before trying again to avoid busy-waiting
            time.sleep(0.01) 
            continue

        cv2.imshow("Frame", frame_to_display)
        processed_frame_queue.task_done() # Indicate that a task from processed_frame_queue is done

        key = cv2.waitKey(1)
        if key == 27: # ESC key to exit
            break

    print("Exiting main display loop.")

    cv2.destroyAllWindows()
    print("Program finished.")

if __name__ == "__main__":
    main()

Frame reader finished.
Frame processor finished.
Exiting main display loop.
Program finished.


In [ ]:
import cv2
import mediapipe as mp
import numpy as np

def is_all_fingers_up(hand_landmarks):
    """
    Checks if all fingers (thumb, index, middle, ring, pinky) are extended upwards
    in a high-five pose.

    Args:
        hand_landmarks: A MediaPipe HandLandmark object containing the 3D coordinates
                        of the hand landmarks.

    Returns:
        True if all fingers are detected as up, False otherwise.
    """

    if not hand_landmarks:
        return False

    # Define landmark indices for each finger and their corresponding MCP, PIP, DIP joints.
    # Note: For the thumb, we primarily check the IP joint relative to the MCP/CMC.
    # For other fingers, we check PIP and DIP relative to MCP.
    fingers = {
        'thumb': [mp_hands.HandLandmark.THUMB_CMC, mp_hands.HandLandmark.THUMB_MCP, mp_hands.HandLandmark.THUMB_IP, mp_hands.HandLandmark.THUMB_TIP],
        'index': [mp_hands.HandLandmark.INDEX_FINGER_MCP, mp_hands.HandLandmark.INDEX_FINGER_PIP, mp_hands.HandLandmark.INDEX_FINGER_DIP, mp_hands.HandLandmark.INDEX_FINGER_TIP],
        'middle': [mp_hands.HandLandmark.MIDDLE_FINGER_MCP, mp_hands.HandLandmark.MIDDLE_FINGER_PIP, mp_hands.HandLandmark.MIDDLE_FINGER_DIP, mp_hands.HandLandmark.MIDDLE_FINGER_TIP],
        'ring': [mp_hands.HandLandmark.RING_FINGER_MCP, mp_hands.HandLandmark.RING_FINGER_PIP, mp_hands.HandLandmark.RING_FINGER_DIP, mp_hands.HandLandmark.RING_FINGER_TIP],
        'pinky': [mp_hands.HandLandmark.PINKY_MCP, mp_hands.HandLandmark.PINKY_PIP, mp_hands.HandLandmark.PINKY_DIP, mp_hands.HandLandmark.PINKY_TIP],
    }

    # A threshold for the angle to determine if a finger is "up" (extended).
    # This might need fine-tuning based on your camera angle and hand size.
    # A smaller angle means more extended.
    angle_threshold = 30  # degrees

    all_fingers_extended = True

    # Get wrist landmark to establish a reference point for "up"
    wrist_y = hand_landmarks.landmark[mp_hands.HandLandmark.WRIST].y

    for finger_name, landmarks in fingers.items():
        if finger_name == 'thumb':
            # For the thumb, check if the tip is significantly above the MCP,
            # and the overall curl.
            mcp = hand_landmarks.landmark[landmarks[1]]
            ip = hand_landmarks.landmark[landmarks[2]]
            tip = hand_landmarks.landmark[landmarks[3]]

            # Check if thumb tip is roughly above or at the same height as the MCP
            # relative to the wrist.
            if tip.y > mcp.y + (wrist_y - mcp.y) * 0.2: # Simple heuristic
                all_fingers_extended = False
                # print(f"{finger_name} not up (y-position check)")
                break

            # Calculate angle between MCP-IP-TIP to check for curl
            # We'll use 2D projection for simplicity, considering x and y
            vec_mcp_ip = np.array([ip.x - mcp.x, ip.y - mcp.y])
            vec_ip_tip = np.array([tip.x - ip.x, tip.y - ip.y])

            dot_product = np.dot(vec_mcp_ip, vec_ip_tip)
            magnitude_mcp_ip = np.linalg.norm(vec_mcp_ip)
            magnitude_ip_tip = np.linalg.norm(vec_ip_tip)

            # Avoid division by zero
            if magnitude_mcp_ip == 0 or magnitude_ip_tip == 0:
                continue

            angle_rad = np.arccos(dot_product / (magnitude_mcp_ip * magnitude_ip_tip))
            angle_deg = np.degrees(angle_rad)

            if angle_deg > 160: # Angle close to 180 means relatively straight
                 # print(f"{finger_name} is curled (angle: {angle_deg})")
                 all_fingers_extended = False
                 break

        else:
            # For other fingers, check PIP and DIP joints relative to MCP.
            # A common approach is to check if the Y-coordinate of the tip is significantly
            # higher than the Y-coordinate of the MCP, and also check joint angles.
            mcp = hand_landmarks.landmark[landmarks[0]]
            pip = hand_landmarks.landmark[landmarks[1]]
            dip = hand_landmarks.landmark[landmarks[2]]
            tip = hand_landmarks.landmark[landmarks[3]]

            # Check if finger is pointing generally upwards (Y-coordinate check)
            # Compare tip's Y with MCP's Y (lower Y means higher in image)
            if tip.y > pip.y or pip.y > mcp.y: # If any joint is lower than previous, it's likely bent down
                all_fingers_extended = False
                # print(f"{finger_name} not up (y-position check)")
                break

            # Calculate angles for PIP and DIP joints
            # PIP joint angle (MCP-PIP-DIP)
            vec_mcp_pip = np.array([pip.x - mcp.x, pip.y - mcp.y, pip.z - mcp.z])
            vec_pip_dip = np.array([dip.x - pip.x, dip.y - pip.y, dip.z - pip.z])
            angle_pip = np.degrees(np.arccos(np.dot(vec_mcp_pip, vec_pip_dip) / (np.linalg.norm(vec_mcp_pip) * np.linalg.norm(vec_pip_dip) + 1e-6)))

            # DIP joint angle (PIP-DIP-TIP)
            vec_pip_dip_2 = np.array([dip.x - pip.x, dip.y - pip.y, dip.z - pip.z]) # Recalculate or reuse
            vec_dip_tip = np.array([tip.x - dip.x, tip.y - dip.y, tip.z - dip.z])
            angle_dip = np.degrees(np.arccos(np.dot(vec_pip_dip_2, vec_dip_tip) / (np.linalg.norm(vec_pip_dip_2) * np.linalg.norm(vec_dip_tip) + 1e-6)))


            if angle_pip > angle_threshold or angle_dip > angle_threshold:
                all_fingers_extended = False
                # print(f"{finger_name} not up (angle check: PIP={angle_pip:.2f}, DIP={angle_dip:.2f})")
                break

    return all_fingers_extended

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

# For webcam input:
cap = cv2.VideoCapture(0)

with mp_hands.Hands(
    min_detection_confidence=0.7,
    min_tracking_confidence=0.5,
    max_num_hands=1) as hands: # Only process one hand for simplicity
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            print("Ignoring empty camera frame.")
            continue

        # Flip the image horizontally for a natural selfie-view display.
        image = cv2.flip(image, 1)

        # To improve performance, optionally mark the image as not writeable to
        # pass by reference.
        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = hands.process(image)

        # Draw the hand annotations on the image.
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        high_five_detected = False
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(
                    image,
                    hand_landmarks,
                    mp_hands.HAND_CONNECTIONS,
                    mp_drawing_styles.get_default_hand_landmarks_style(),
                    mp_drawing_styles.get_default_hand_connections_style())

                if is_all_fingers_up(hand_landmarks):
                    high_five_detected = True
                    break # Only need one hand for high five

        # Display detection status
        if high_five_detected:
            cv2.putText(image, "HIGH FIVE!", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 255, 0), 3, cv2.LINE_AA)
        else:
            cv2.putText(image, "No High Five", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 0, 255), 3, cv2.LINE_AA)


        cv2.imshow('MediaPipe Hands - High Five Detector', image)
        if cv2.waitKey(5) & 0xFF == 27:
            break
cap.release()
cv2.destroyAllWindows()